In [1]:
import pandas as pd
import numpy as np
from time import time
from config_loader import load
import argparse
import sys
import seaborn as sns
from MyDataUnderstanding import featureAnalysis
from MyPreprocessing import MyPreprocessing
import numpy as np
from time import time
import matplotlib.pyplot as plt
from model.models import models_perform
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from statistics import mean
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from detect_outliers import detect_outliers
from MyFeatureSelection import MyFeatureSelection

In [2]:
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.svm import SVC
from xgboost import XGBClassifier as XGB
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from model.MyIBL import MyIBL as IBL
from sklearn.linear_model import LogisticRegression as LG

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)

In [59]:
def getData(path, filenames_type):
    '''
    features_lst = [
        "Pclass", "Survived", "Name", "Sex", "Age",
        "Sibsp", "Parch", "Ticket", "Fare", "Cabin","Embarked"]
    '''
    if filenames_type == 'train':
        filename = 'train'
    elif filenames_type == 'test':
        filename = 'test'
    else:
        filename = 'titanicAll'
    df_features = pd.read_csv(path + filename + '.csv',
                           sep=',')

    if filenames_type not in ['train', 'test']:
        # drop unnecessary columns that don't exist in the official dataset
        df_features.drop(['Boat', 'Body', 'Home.dest'],
                          axis=1,
                         inplace=True)
    #labels = df_features['Survived']
    #df_features = df_features.drop(['Survived'], axis=1)
    return df_features

In [60]:
##
# Loads config
parser = argparse.ArgumentParser()
parser.add_argument(
    "-c", "--config", default="titanic.cfg",
    help="specify the location of the clustering config file"
)
args, _ = parser.parse_known_args()

config_file = args.config
config = load(config_file)

##
verbose = config.get('titanic', 'verbose')
path = config.get('titanic', 'path') + '/'
file_type = config.get('titanic', 'file_type')

filename_type = 'train'
if file_type == 'all':
    filename_type = 'other'

In [61]:
print('Filename type:', filename_type)
print()
## train
trainData = getData(path, filename_type)
# Preprocessing
trainPreprocess = MyPreprocessing(process_type='all',
                                  filename_type=filename_type,
                                  remove_outliers=True)

## test
filename_type = 'test'
testData = getData(path, filename_type)
# Preprocessing
testPreprocess = MyPreprocessing(process_type='all',
                                 filename_type=filename_type,
                                 remove_outliers=False)


Filename type: train



In [62]:
trainPreprocess.fit(trainData)
df_train = trainPreprocess.new_df
# the labels "Survived"
labels = trainPreprocess.labels_
#print(labels.head())
# the initial dataset without any preprocessing
#print(trainPreprocess.df_initial.head())
# the preprocessed data
#print(trainPreprocess.new_df.head())

testPreprocess.fit(testData)
df_test = testPreprocess.new_df

process_type: all
Number of outliers 25
process_type: all


In [63]:
# fix missing columns because of NaNs and one hot encoding without dummy_na
if df_train.shape[1] != df_test.shape[1]:
    missing_cols = set(df_test.columns) - set(df_train.columns)
    for col in missing_cols:
        #df_train[col] = np.zeros([df_train.shape[0], 1])
        df_test.drop([col], axis=1, inplace=True)

    missing_cols = set(df_train.columns) - set(df_test.columns)
    for col in missing_cols:
        #df_test[col] = np.zeros([df_test.shape[0], 1])
        df_train.drop([col], axis=1, inplace=True)

labels_test = testPreprocess.labels_

print(df_train.columns, df_test.columns)
print(df_train.shape, df_test.shape)

Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'PclassCp_2', 'PclassCp_3', 'Title_Mr0', 'Title_Mr50', 'Title_Mrs', 'Title_Ms', 'FamilySize', 'Em_C', 'Em_Q', 'Em_S', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_X', 'Age_bin_Kid', 'Age_bin_Teenager', 'Age_bin_Adult', 'Family_bin_SmallFamily', 'Family_bin_BigFamily', 'Family_bin_Team', 'Fare_bin_Median', 'Fare_bin_Average', 'Fare_bin_High', 'Mother', 'Father', 'Daughter', 'Son', 'Orphan', 'RichWoman', 'MiddleClassWoman', 'PoorWoman', 'RichMan', 'MiddleClassMan', 'PoorMan', 'RichGirl', 'MiddleClassGirl', 'PoorGirl', 'RichBoy', 'MiddleClassBoy', 'PoorBoy'], dtype='object') Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'PclassCp_2', 'PclassCp_3', 'Title_Mr0', 'Title_Mr50', 'Title_Mrs', 'Title_Ms', 'FamilySize', 'Em_C', 'Em_Q', 'Em_S', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_X', 'Age_bin_Kid', 'Age_bin_Teenager', 'Age_bin_Adult', 'Family_bin_SmallFamily', 'Fam

In [64]:
df_train.columns

Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'PclassCp_2', 'PclassCp_3', 'Title_Mr0', 'Title_Mr50', 'Title_Mrs', 'Title_Ms', 'FamilySize', 'Em_C', 'Em_Q', 'Em_S', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_X', 'Age_bin_Kid', 'Age_bin_Teenager', 'Age_bin_Adult', 'Family_bin_SmallFamily', 'Family_bin_BigFamily', 'Family_bin_Team', 'Fare_bin_Median', 'Fare_bin_Average', 'Fare_bin_High', 'Mother', 'Father', 'Daughter', 'Son', 'Orphan', 'RichWoman', 'MiddleClassWoman', 'PoorWoman', 'RichMan', 'MiddleClassMan', 'PoorMan', 'RichGirl', 'MiddleClassGirl', 'PoorGirl', 'RichBoy', 'MiddleClassBoy', 'PoorBoy'], dtype='object')

In [65]:
df_test.columns

Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'PclassCp_2', 'PclassCp_3', 'Title_Mr0', 'Title_Mr50', 'Title_Mrs', 'Title_Ms', 'FamilySize', 'Em_C', 'Em_Q', 'Em_S', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_X', 'Age_bin_Kid', 'Age_bin_Teenager', 'Age_bin_Adult', 'Family_bin_SmallFamily', 'Family_bin_BigFamily', 'Family_bin_Team', 'Fare_bin_Median', 'Fare_bin_Average', 'Fare_bin_High', 'Mother', 'Father', 'Daughter', 'Son', 'Orphan', 'RichWoman', 'MiddleClassWoman', 'PoorWoman', 'RichMan', 'MiddleClassMan', 'PoorMan', 'RichGirl', 'MiddleClassGirl', 'PoorGirl', 'RichBoy', 'MiddleClassBoy', 'PoorBoy'], dtype='object')

In [11]:
#df_test.dtypes

In [66]:
def feature_importance(clf, df_train):
    if hasattr(clf, 'feature_importances_'):
        name = str(clf.__class__).split('.')[-1][:-2]
        feat_imp = pd.DataFrame({'importance':clf.feature_importances_})    
        feat_imp['feature'] = df_train.columns
        feat_imp.sort_values(by='importance', ascending=False, inplace=True)
        #feat_imp = feat_imp.iloc[:top_n]
        feat_imp.sort_values(by='importance', inplace=True)
        feat_imp = feat_imp.set_index('feature', drop=True)
        feat_imp.plot.barh(title='Feature Importance', figsize=(10,10))
        plt.xlabel('%s Feature Importance Score' %name)
        plt.show()
        return feat_imp
    return pd.DataFrame()

In [67]:
def ensemble(clf, x_train, y_train, x_test):
    
    name = str(clf.__class__).split('.')[-1][:-2]
    kf = KFold(n_splits=5)

    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]

    #ens_train = np.zeros((ntrain,))
    ens_train = np.array([])
    #ens_test = np.zeros((ntest,))
    #test_kf = np.empty((kf.n_folds, ntest))
    test_kf = np.zeros((kf.n_splits, ntest))
        
    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        clf.fit(x_tr, y_tr)

        #ens_train[test_index] = clf.predict(x_te)
        ens_train = np.concatenate([ens_train, clf.predict(x_te)])
        # predict from the whole train set 
        #clf.fit(x_train,y_train)
        test_kf[i, :] = clf.predict(x_test)
    
    # features for next ensemble layer
    ens_train = pd.DataFrame(ens_train.reshape(-1,1), columns=[name])
    
    # combine for test
    ens_test = pd.DataFrame(test_kf.T).mode(axis=1)
    ens_test.columns = [name]
    
    return ens_train, ens_test

In [68]:
def modeler(clf, data, labels, train_fidx, validation_fidx, name_prefix):
    name = name_prefix + str(clf.__class__).split('.')[-1][:-2]
    #print(name)
    folds_accuracy = list()
    duration = list()
    start = time()
    for idx, trf in enumerate(train_fidx):
        clf.fit(data.loc[trf], labels.loc[trf])
        prediction_labels = clf.predict(data.loc[validation_fidx[idx]])
        folds_accuracy.append(accuracy_score(labels.loc[validation_fidx[idx]], prediction_labels))
        
    mean_acc = mean(folds_accuracy)
    duration = time() - start
    #print(name, mean_acc, "accuracy at validation stage,", duration, 's')
    df_folds_accuracy = pd.DataFrame([folds_accuracy], index=[name])
    df_folds = pd.DataFrame([[mean_acc, duration/len(train_fidx)]], columns=['Accuracy', 'Duration'], index=[name])
    return clf.fit(data, labels), df_folds, df_folds_accuracy

In [69]:
def get_cv_data(df_train, cv=5):
    kf = KFold(n_splits=cv)
    folds = [(train_idx, validation_idx) for train_idx, validation_idx in kf.split(df_train)]
    train_idx = [f[0] for f in folds]
    validation_idx = [f[1] for f in folds]
    return train_idx, validation_idx

In [70]:
def run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True):
    models = []
    df_ens_features = pd.DataFrame()
    df_ens_test = pd.DataFrame()
    df_model_cvaccuracies = pd.DataFrame()
    df_model_info = pd.DataFrame()
    for clf in clfs:
        name = str(clf.__class__).split('.')[-1][:-2]
        
        # cross validation
        if cv:
            model, df_folds, df_folds_accuracy = modeler(clf, df_train, labels, train_idx, validation_idx, name_prefix)
            models.append(model)
            df_model_cvaccuracies = pd.concat([df_model_cvaccuracies, df_folds_accuracy], sort=False)
            df_model_info = pd.concat([df_model_info, df_folds], sort=False)  
            
            #feature_importance(clf, df_train)
        
        # building ensemble
        if cv and ens:
            #df_feature = pd.DataFrame(cross_val_predict(clf, df_train, labels, cv=ens_kf).reshape(-1,1), columns=[name])
            df_ens_feature, ens_test = ensemble(clf, df_train, labels, df_test)
            df_ens_features = pd.concat([df_ens_features, df_ens_feature], axis=1, sort=False).reset_index(drop=True)
            df_ens_test = pd.concat([df_ens_test, ens_test], axis=1, sort=False).reset_index(drop=True)
            
    return models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test
        

In [71]:
clfs = [IBL(), RF(n_estimators =12), SVC(gamma='scale'), XGB(), MLP(max_iter=1000), KNN(), LDA()]
#clfs = [RF(n_estimators =12), SVC(gamma='scale'), XGB(), KNN(), LDA()]

In [72]:
train_idx, validation_idx = get_cv_data(df_train, cv=5)

In [73]:
import warnings
warnings.filterwarnings("ignore")

In [74]:
print('Original')
print('##################################')
name_prefix = 'ALL_' + str(len(df_train.columns)) + '_'
models, df_results_all_cv, df_results_all, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
df_results_all

Original
##################################


,Accuracy,Duration
ALL_49_MyIBL,0.750575,2.488669
ALL_49_RandomForestClassifier,0.800252,0.027988
ALL_49_SVC,0.821009,0.036744
ALL_49_XGBClassifier,0.822198,0.148959
ALL_49_MLPClassifier,0.804877,4.068736
ALL_49_KNeighborsClassifier,0.807196,0.014191
ALL_49_LinearDiscriminantAnalysis,0.814099,0.018538


In [75]:
print('PCA')
print('##################################')
df_results_pca = pd.DataFrame()
df_results_pca_cv = pd.DataFrame()
for n_dim in range(15, 22):
    print(n_dim, ' dimensions')
    pca_train, pca_test, ev = MyFeatureSelection.applyPCA(df_train, df_test, n_dim)
    name_prefix = 'PCA_d_' + str(n_dim) + '/' + str(len(df_train.columns)) + '_ev_' + str(round(ev,3)) + '_'
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_pca_cv = pd.concat([df_results_pca_cv, df_model_cvaccuracies], axis=0)
    df_results_pca = pd.concat([df_results_pca, df_model_info], axis=0)

PCA
##################################
15  dimensions
16  dimensions
17  dimensions
18  dimensions
19  dimensions
20  dimensions
21  dimensions


In [76]:
df_results_pca

,Accuracy,Duration
PCA_d_15/49_ev_0.871_MyIBL,0.749452,2.459888
PCA_d_15/49_ev_0.871_RandomForestClassifier,0.815268,0.024192
PCA_d_15/49_ev_0.871_SVC,0.821009,0.031896
PCA_d_15/49_ev_0.871_XGBClassifier,0.822198,0.137252
PCA_d_15/49_ev_0.871_MLPClassifier,0.804903,4.294611
PCA_d_15/49_ev_0.871_KNeighborsClassifier,0.807196,0.013792
PCA_d_15/49_ev_0.871_LinearDiscriminantAnalysis,0.814099,0.017589
PCA_d_16/49_ev_0.885_MyIBL,0.739120,2.454343
PCA_d_16/49_ev_0.885_RandomForestClassifier,0.801382,0.022986
PCA_d_16/49_ev_0.885_SVC,0.821009,0.030781


In [77]:
df_results_pca_order = df_results_pca.copy()
df_results_pca_order.sort_values('Accuracy')

,Accuracy,Duration
PCA_d_16/49_ev_0.885_MyIBL,0.739120,2.454343
PCA_d_21/49_ev_0.941_MyIBL,0.741386,2.523447
PCA_d_17/49_ev_0.898_MyIBL,0.742555,2.516052
PCA_d_18/49_ev_0.91_MyIBL,0.747193,2.478875
PCA_d_15/49_ev_0.871_MyIBL,0.749452,2.459888
PCA_d_20/49_ev_0.932_MyIBL,0.749465,2.469081
PCA_d_19/49_ev_0.922_MyIBL,0.759810,2.487470
PCA_d_16/49_ev_0.885_MLPClassifier,0.795648,4.294456
PCA_d_21/49_ev_0.941_MLPClassifier,0.796778,4.176484
PCA_d_18/49_ev_0.91_MLPClassifier,0.796791,4.277292


In [78]:
df_results_pca_cv

,0,1,2,3,4
PCA_d_15/49_ev_0.871_MyIBL,0.724138,0.722543,0.763006,0.774566,0.763006
PCA_d_15/49_ev_0.871_RandomForestClassifier,0.793103,0.791908,0.843931,0.815029,0.832370
PCA_d_15/49_ev_0.871_SVC,0.827586,0.809249,0.815029,0.797688,0.855491
PCA_d_15/49_ev_0.871_XGBClassifier,0.798851,0.815029,0.832370,0.797688,0.867052
PCA_d_15/49_ev_0.871_MLPClassifier,0.758621,0.786127,0.826590,0.803468,0.849711
PCA_d_15/49_ev_0.871_KNeighborsClassifier,0.775862,0.803468,0.820809,0.809249,0.826590
PCA_d_15/49_ev_0.871_LinearDiscriminantAnalysis,0.804598,0.815029,0.809249,0.791908,0.849711
PCA_d_16/49_ev_0.885_MyIBL,0.660920,0.710983,0.791908,0.757225,0.774566
PCA_d_16/49_ev_0.885_RandomForestClassifier,0.804598,0.780347,0.820809,0.774566,0.826590
PCA_d_16/49_ev_0.885_SVC,0.827586,0.809249,0.815029,0.797688,0.855491


In [79]:
max(df_results_pca['Accuracy'])

0.8221978606072686

In [80]:
print('ICA')
print('##################################')
df_results_ica = pd.DataFrame()
df_results_ica_cv = pd.DataFrame()
for n_dim in [25,30,35,40,45]:
    print(n_dim, ' dimensions')
    ica_train, ica_test = MyFeatureSelection.applyICA(df_train, df_test, n_dim)
    name_prefix = 'ICA_d_' + str(n_dim) + '/' + str(len(df_train.columns))
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_ica_cv = pd.concat([df_results_ica_cv, df_model_cvaccuracies], axis=0)
    df_results_ica = pd.concat([df_results_ica, df_model_info], axis=0)

ICA
##################################
25  dimensions
30  dimensions
35  dimensions
40  dimensions
45  dimensions


In [81]:
df_results_ica

,Accuracy,Duration
ICA_d_25/49MyIBL,0.763291,2.470292
ICA_d_25/49RandomForestClassifier,0.817587,0.023186
ICA_d_25/49SVC,0.821009,0.030781
ICA_d_25/49XGBClassifier,0.822198,0.137316
ICA_d_25/49MLPClassifier,0.794479,3.841027
ICA_d_25/49KNeighborsClassifier,0.807196,0.013392
ICA_d_25/49LinearDiscriminantAnalysis,0.814099,0.017286
ICA_d_30/49MyIBL,0.755245,2.489069
ICA_d_30/49RandomForestClassifier,0.814112,0.023386
ICA_d_30/49SVC,0.821009,0.030981


In [82]:
df_results_ica_cv

,0,1,2,3,4
ICA_d_25/49MyIBL,0.752874,0.739884,0.739884,0.791908,0.791908
ICA_d_25/49RandomForestClassifier,0.787356,0.803468,0.838150,0.797688,0.861272
ICA_d_25/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491
ICA_d_25/49XGBClassifier,0.798851,0.815029,0.832370,0.797688,0.867052
ICA_d_25/49MLPClassifier,0.775862,0.774566,0.791908,0.786127,0.843931
ICA_d_25/49KNeighborsClassifier,0.775862,0.803468,0.820809,0.809249,0.826590
ICA_d_25/49LinearDiscriminantAnalysis,0.804598,0.815029,0.809249,0.791908,0.849711
ICA_d_30/49MyIBL,0.712644,0.739884,0.768786,0.803468,0.751445
ICA_d_30/49RandomForestClassifier,0.793103,0.791908,0.838150,0.809249,0.838150
ICA_d_30/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491


In [83]:
df_results_ica_order = df_results_ica.copy()
df_results_ica_order.sort_values('Accuracy')

,Accuracy,Duration
ICA_d_35/49MyIBL,0.727520,2.471080
ICA_d_40/49MyIBL,0.734463,2.494464
ICA_d_45/49MyIBL,0.737944,2.544272
ICA_d_30/49MyIBL,0.755245,2.489069
ICA_d_25/49MyIBL,0.763291,2.470292
ICA_d_25/49MLPClassifier,0.794479,3.841027
ICA_d_35/49RandomForestClassifier,0.795608,0.023786
ICA_d_30/49MLPClassifier,0.797927,4.617026
ICA_d_35/49MLPClassifier,0.797954,4.062485
ICA_d_40/49RandomForestClassifier,0.800246,0.026403


In [84]:
max(df_results_ica['Accuracy'])

0.8221978606072686

In [85]:
print('INFO GAIN SELECTION')
print('##################################')
df_results_ig = pd.DataFrame()
df_results_ig_cv = pd.DataFrame()
for n_dim in [25,30,35,40,45]:
    print(n_dim, ' dimensions')
    ig_train, ig_test = MyFeatureSelection.InfoGainSelection(df_train, df_test, labels, n_dim)
    name_prefix = 'IG_d_' + str(n_dim) + '/' + str(len(df_train.columns))
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_ig_cv = pd.concat([df_results_ig_cv, df_model_cvaccuracies], axis=0)
    df_results_ig = pd.concat([df_results_ig, df_model_info], axis=0)

INFO GAIN SELECTION
##################################
25  dimensions
30  dimensions
35  dimensions
40  dimensions
45  dimensions


In [86]:
df_results_ig

,Accuracy,Duration
IG_d_25/49MyIBL,0.747100,2.544893
IG_d_25/49RandomForestClassifier,0.817547,0.025476
IG_d_25/49SVC,0.821009,0.032788
IG_d_25/49XGBClassifier,0.822198,0.138954
IG_d_25/49MLPClassifier,0.797940,4.061126
IG_d_25/49KNeighborsClassifier,0.807196,0.014791
IG_d_25/49LinearDiscriminantAnalysis,0.814099,0.018088
IG_d_30/49MyIBL,0.764454,2.658309
IG_d_30/49RandomForestClassifier,0.802571,0.030383
IG_d_30/49SVC,0.821009,0.033194


In [87]:
df_results_ig_order = df_results_ig.copy()
df_results_ig_order.sort_values('Accuracy')

,Accuracy,Duration
IG_d_40/49MyIBL,0.735626,2.668113
IG_d_25/49MyIBL,0.747100,2.544893
IG_d_45/49MyIBL,0.762135,2.631687
IG_d_30/49MyIBL,0.764454,2.658309
IG_d_35/49MyIBL,0.769098,2.670627
IG_d_25/49MLPClassifier,0.797940,4.061126
IG_d_40/49MLPClassifier,0.800266,3.899812
IG_d_35/49MLPClassifier,0.801395,4.280081
IG_d_45/49RandomForestClassifier,0.801402,0.025320
IG_d_30/49MLPClassifier,0.801422,3.985469


In [88]:
df_results_ig_cv

,0,1,2,3,4
IG_d_25/49MyIBL,0.758621,0.699422,0.768786,0.739884,0.768786
IG_d_25/49RandomForestClassifier,0.821839,0.803468,0.843931,0.780347,0.838150
IG_d_25/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491
IG_d_25/49XGBClassifier,0.798851,0.815029,0.832370,0.797688,0.867052
IG_d_25/49MLPClassifier,0.781609,0.780347,0.791908,0.791908,0.843931
IG_d_25/49KNeighborsClassifier,0.775862,0.803468,0.820809,0.809249,0.826590
IG_d_25/49LinearDiscriminantAnalysis,0.804598,0.815029,0.809249,0.791908,0.849711
IG_d_30/49MyIBL,0.747126,0.757225,0.728324,0.809249,0.780347
IG_d_30/49RandomForestClassifier,0.775862,0.791908,0.820809,0.786127,0.838150
IG_d_30/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491


In [89]:
max(df_results_ig['Accuracy'])

0.8221978606072686

In [90]:
print('ANOVA SELECTION')
print('##################################')
df_results_an = pd.DataFrame()
df_results_an_cv = pd.DataFrame()
for n_dim in [25,30,35,40,45]:
    print(n_dim, ' dimensions')
    an_train, an_test = MyFeatureSelection.AnovaSelection(df_train, df_test, labels, n_dim)
    name_prefix = 'AN_d_' + str(n_dim) + '/' + str(len(df_train.columns))
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_an_cv = pd.concat([df_results_an_cv, df_model_cvaccuracies], axis=0)
    df_results_an = pd.concat([df_results_an, df_model_info], axis=0)

ANOVA SELECTION
##################################
25  dimensions
30  dimensions
35  dimensions
40  dimensions
45  dimensions


In [91]:
df_results_an

,Accuracy,Duration
AN_d_25/49MyIBL,0.725214,2.759499
AN_d_25/49RandomForestClassifier,0.799090,0.023931
AN_d_25/49SVC,0.821009,0.031793
AN_d_25/49XGBClassifier,0.822198,0.134518
AN_d_25/49MLPClassifier,0.800246,4.487139
AN_d_25/49KNeighborsClassifier,0.807196,0.013991
AN_d_25/49LinearDiscriminantAnalysis,0.814099,0.017827
AN_d_30/49MyIBL,0.745970,2.562977
AN_d_30/49RandomForestClassifier,0.810664,0.023795
AN_d_30/49SVC,0.821009,0.031790


In [92]:
df_results_an_order = df_results_an.copy()
df_results_an_order.sort_values('Accuracy')

,Accuracy,Duration
AN_d_45/49MyIBL,0.725188,2.563406
AN_d_25/49MyIBL,0.725214,2.759499
AN_d_30/49MyIBL,0.745970,2.562977
AN_d_35/49MyIBL,0.763351,2.438941
AN_d_40/49MyIBL,0.776002,2.575579
AN_d_35/49RandomForestClassifier,0.795595,0.024411
AN_d_45/49RandomForestClassifier,0.797934,0.023995
AN_d_25/49RandomForestClassifier,0.799090,0.023931
AN_d_40/49MLPClassifier,0.799116,4.808649
AN_d_45/49MLPClassifier,0.800246,4.167284


In [93]:
df_results_an_cv

,0,1,2,3,4
AN_d_25/49MyIBL,0.689655,0.699422,0.751445,0.734104,0.751445
AN_d_25/49RandomForestClassifier,0.787356,0.786127,0.838150,0.780347,0.803468
AN_d_25/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491
AN_d_25/49XGBClassifier,0.798851,0.815029,0.832370,0.797688,0.867052
AN_d_25/49MLPClassifier,0.787356,0.768786,0.797688,0.797688,0.849711
AN_d_25/49KNeighborsClassifier,0.775862,0.803468,0.820809,0.809249,0.826590
AN_d_25/49LinearDiscriminantAnalysis,0.804598,0.815029,0.809249,0.791908,0.849711
AN_d_30/49MyIBL,0.735632,0.722543,0.763006,0.786127,0.722543
AN_d_30/49RandomForestClassifier,0.775862,0.803468,0.832370,0.809249,0.832370
AN_d_30/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491


In [94]:
max(df_results_an['Accuracy'])

0.8221978606072686

In [95]:
# Random forest estimator for feature selection
print('Random Forest Estimator Selection')
print('##################################')
rf_train, rf_test = MyFeatureSelection.RandomForestSelection(df_train, df_test, labels, 100)
print(len(rf_train.columns), ' dimensions')
name_prefix = 'RF_d_' + str(len(rf_train.columns)) + '/' + str(len(df_train.columns))
models, df_results_rf_cv, df_results_rf, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)



Random Forest Estimator Selection
##################################
25  dimensions


In [96]:
df_results_rf

,Accuracy,Duration
RF_d_25/49MyIBL,0.778340,2.562606
RF_d_25/49RandomForestClassifier,0.802565,0.024193
RF_d_25/49SVC,0.821009,0.031594
RF_d_25/49XGBClassifier,0.822198,0.134884
RF_d_25/49MLPClassifier,0.799103,4.252314
RF_d_25/49KNeighborsClassifier,0.807196,0.013991
RF_d_25/49LinearDiscriminantAnalysis,0.814099,0.017949


In [97]:
df_results_rf_cv

,0,1,2,3,4
RF_d_25/49MyIBL,0.735632,0.768786,0.815029,0.774566,0.797688
RF_d_25/49RandomForestClassifier,0.781609,0.791908,0.826590,0.791908,0.820809
RF_d_25/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491
RF_d_25/49XGBClassifier,0.798851,0.815029,0.832370,0.797688,0.867052
RF_d_25/49MLPClassifier,0.775862,0.780347,0.809249,0.797688,0.832370
RF_d_25/49KNeighborsClassifier,0.775862,0.803468,0.820809,0.809249,0.826590
RF_d_25/49LinearDiscriminantAnalysis,0.804598,0.815029,0.809249,0.791908,0.849711


In [98]:
print('Lasso Regression Selection')
print('##################################')
df_results_lr = pd.DataFrame()
df_results_lr_cv = pd.DataFrame()
for alpha in [0.003, 0.0009, 0.0005, 0.0002, 0.00001]:
    lr_train, lr_test = MyFeatureSelection.LassoRegressionSelection(df_train, df_test, labels, alpha)
    print(len(lr_train.columns), ' dimensions')
    name_prefix = 'LR_d_' + str(n_dim) + '/' + str(len(df_train.columns))
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_lr_cv = pd.concat([df_results_lr_cv, df_model_cvaccuracies], axis=0)
    df_results_lr = pd.concat([df_results_lr, df_model_info], axis=0)

Lasso Regression Selection
##################################
18  dimensions
29  dimensions
37  dimensions
40  dimensions
45  dimensions


In [99]:
df_results_lr

,Accuracy,Duration
LR_d_45/49MyIBL,0.733280,2.619252
LR_d_45/49RandomForestClassifier,0.799110,0.024299
LR_d_45/49SVC,0.821009,0.031914
LR_d_45/49XGBClassifier,0.822198,0.136725
LR_d_45/49MLPClassifier,0.801402,4.764713
LR_d_45/49KNeighborsClassifier,0.807196,0.013998
LR_d_45/49LinearDiscriminantAnalysis,0.814099,0.017656
LR_d_45/49MyIBL,0.744781,2.548061
LR_d_45/49RandomForestClassifier,0.795615,0.025784
LR_d_45/49SVC,0.821009,0.032173


In [100]:
df_results_lr_cv

,0,1,2,3,4
LR_d_45/49MyIBL,0.712644,0.710983,0.716763,0.757225,0.768786
LR_d_45/49RandomForestClassifier,0.770115,0.780347,0.826590,0.797688,0.820809
LR_d_45/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491
LR_d_45/49XGBClassifier,0.798851,0.815029,0.832370,0.797688,0.867052
LR_d_45/49MLPClassifier,0.787356,0.780347,0.803468,0.791908,0.843931
LR_d_45/49KNeighborsClassifier,0.775862,0.803468,0.820809,0.809249,0.826590
LR_d_45/49LinearDiscriminantAnalysis,0.804598,0.815029,0.809249,0.791908,0.849711
LR_d_45/49MyIBL,0.764368,0.699422,0.751445,0.780347,0.728324
LR_d_45/49RandomForestClassifier,0.793103,0.791908,0.809249,0.791908,0.791908
LR_d_45/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491


In [101]:
df_results_lr_order = df_results_lr.copy()
df_results_lr_order.sort_values('Accuracy')

,Accuracy,Duration
LR_d_45/49MyIBL,0.721779,2.635361
LR_d_45/49MyIBL,0.730995,2.590637
LR_d_45/49MyIBL,0.733280,2.619252
LR_d_45/49MyIBL,0.741346,2.549946
LR_d_45/49MyIBL,0.744781,2.548061
LR_d_45/49RandomForestClassifier,0.795615,0.025784
LR_d_45/49MLPClassifier,0.797947,4.684834
LR_d_45/49RandomForestClassifier,0.799110,0.024299
LR_d_45/49MLPClassifier,0.800252,4.136561
LR_d_45/49MLPClassifier,0.800259,3.683260


In [102]:
df_total_order =  pd.concat([df_results_all, df_results_pca, df_results_ica, df_results_ig, df_results_an, df_results_rf, df_results_lr], axis=0)
df_total_order.sort_values('Accuracy')

,Accuracy,Duration
LR_d_45/49MyIBL,0.721779,2.635361
AN_d_45/49MyIBL,0.725188,2.563406
AN_d_25/49MyIBL,0.725214,2.759499
ICA_d_35/49MyIBL,0.727520,2.471080
LR_d_45/49MyIBL,0.730995,2.590637
LR_d_45/49MyIBL,0.733280,2.619252
ICA_d_40/49MyIBL,0.734463,2.494464
IG_d_40/49MyIBL,0.735626,2.668113
ICA_d_45/49MyIBL,0.737944,2.544272
PCA_d_16/49_ev_0.885_MyIBL,0.739120,2.454343


In [104]:
lr_train.shape

(866, 45)